In [1]:
import datetime

import drms
import astropy.units as u
from sunpy.net import Fido, attrs as a
from sunpy.map import Map

In [2]:
tr = a.Time("2017-09-04 00:00:00.000", "2017-09-08 00:00:00.000")
result = Fido.search(tr,
                      a.hek.FL,
                      a.hek.FL.GOESCls > "X1.0",
                      a.hek.OBS.Observatory == "GOES")

In [3]:
hek_results = result["hek"]["event_starttime", "event_peaktime",
                             "event_endtime", "fl_goescls", "ar_noaanum"]
hek_results

event_starttime,event_peaktime,event_endtime,fl_goescls,ar_noaanum
Time,Time,Time,str4,int64
2017-09-06 08:57:00.000,2017-09-06 09:10:00.000,2017-09-06 09:17:00.000,X2.2,12673
2017-09-06 11:53:00.000,2017-09-06 12:02:00.000,2017-09-06 12:10:00.000,X9.3,12673
2017-09-07 14:20:00.000,2017-09-07 14:36:00.000,2017-09-07 14:55:00.000,X1.3,12673


In [4]:
peaktime= hek_results["event_peaktime"]
peaktime

<Time object: scale='utc' format='iso' value=['2017-09-06 09:10:00.000' '2017-09-06 12:02:00.000'
 '2017-09-07 14:36:00.000']>

In [5]:
times = []

time_interval = datetime.timedelta(minutes=30)

for pt in peaktime:
    times.append(a.Time(pt - time_interval, pt + time_interval))

In [6]:
times

[<sunpy.net.attrs.Time(2017-09-06 08:40:00.000, 2017-09-06 09:40:00.000)>,
 <sunpy.net.attrs.Time(2017-09-06 11:32:00.000, 2017-09-06 12:32:00.000)>,
 <sunpy.net.attrs.Time(2017-09-07 14:06:00.000, 2017-09-07 15:06:00.000)>]

# SHARP

In [7]:
c = drms.Client(email="mgjeon@khu.ac.kr")

In [8]:
c.info("hmi.sharp_cea_720s").segments

,type,units,protocol,dims,note
name,,,,,
magnetogram,int,Gauss,fits,VARxVAR,Line-of-sight magnetogram in CEA projection
bitmap,char,Enumerated,fits,VARxVAR,Mask for the patch in CEA coordinates
Dopplergram,int,m/s,fits,VARxVAR,Dopplergram in CEA projection
continuum,int,DN/s,fits,VARxVAR,Intensitygram in CEA projection
Bp,int,Gauss,fits,VARxVAR,"B_phi, positive westward"
Bt,int,Gauss,fits,VARxVAR,"B_theta, positive southward"
Br,int,Gauss,fits,VARxVAR,"B_r, positive up"
Bp_err,int,Gauss,fits,VARxVAR,Standard deviation of B_phi
Bt_err,int,Gauss,fits,VARxVAR,Standard deviation of B_theta


In [9]:
c.pkeys("hmi.sharp_cea_720s")

['HARPNUM', 'T_REC']

In [10]:
find_ar = c.query("hmi.Mharp_720s[][%sZ]" % peaktime[0],
                   key = ["NOAA_AR", "HARPNUM"])
find_ar

,NOAA_AR,HARPNUM
0,12673,7115
1,12674,7117
2,0,7118
3,12677,7122
4,12675,7123
5,0,7125
6,12678,7127
7,0,7128
8,0,7129


In [12]:
hmi_results = Fido.search(
    tr,
    a.jsoc.Notify("mgjeon@khu.ac.kr"),
    a.jsoc.Series("hmi.sharp_cea_720s"),
    a.jsoc.PrimeKey('HARPNUM', '7115'),
    a.jsoc.Segment("Bp") & a.jsoc.Segment("Bt") & a.jsoc.Segment("Br"),
)

In [14]:
hmi_files = Fido.fetch(hmi_results, path="download/12673/hmi/{file}")

Export request pending. [id=JSOC_20240405_000216, status=2]
Waiting for 0 seconds...
Export request pending. [id=JSOC_20240405_000216, status=1]
Waiting for 5 seconds...
Export request pending. [id=JSOC_20240405_000216, status=1]
Waiting for 5 seconds...
Export request pending. [id=JSOC_20240405_000216, status=1]
Waiting for 5 seconds...
Export request pending. [id=JSOC_20240405_000216, status=1]
Waiting for 5 seconds...
Export request pending. [id=JSOC_20240405_000216, status=1]
Waiting for 5 seconds...
Export request pending. [id=JSOC_20240405_000216, status=1]
Waiting for 5 seconds...
Export request pending. [id=JSOC_20240405_000216, status=1]
Waiting for 5 seconds...
Export request pending. [id=JSOC_20240405_000216, status=1]
Waiting for 5 seconds...
Export request pending. [id=JSOC_20240405_000216, status=1]
Waiting for 5 seconds...
Export request pending. [id=JSOC_20240405_000216, status=1]
Waiting for 5 seconds...
Export request pending. [id=JSOC_20240405_000216, status=1]
Waiti

Files Downloaded:   0%|          | 0/1299 [00:00<?, ?file/s]

hmi.sharp_cea_720s.7115.20170904_000000_TAI.Bp.fits:   0%|          | 0.00/590k [00:00<?, ?B/s]

hmi.sharp_cea_720s.7115.20170904_000000_TAI.Bt.fits:   0%|          | 0.00/579k [00:00<?, ?B/s]

hmi.sharp_cea_720s.7115.20170904_000000_TAI.Br.fits:   0%|          | 0.00/539k [00:00<?, ?B/s]

...

hmi.sharp_cea_720s.7115.20170907_234800_TAI.Bt.fits:   0%|          | 0.00/582k [00:00<?, ?B/s]

hmi.sharp_cea_720s.7115.20170907_234800_TAI.Br.fits:   0%|          | 0.00/579k [00:00<?, ?B/s]

hmi.sharp_cea_720s.7115.20170908_000000_TAI.Bp.fits:   0%|          | 0.00/547k [00:00<?, ?B/s]

hmi.sharp_cea_720s.7115.20170908_000000_TAI.Bt.fits:   0%|          | 0.00/582k [00:00<?, ?B/s]

hmi.sharp_cea_720s.7115.20170908_000000_TAI.Br.fits:   0%|          | 0.00/579k [00:00<?, ?B/s]

# AIA

In [15]:
c.info("aia.lev1_euv_12s").segments

,type,units,protocol,dims,note
name,,,,,
image,int,None,link via lev1,None,AIA level 1 image
spikes,int,None,link via lev1,None,Cosmic ray information


In [16]:
c.pkeys("aia.lev1_euv_12s")

['T_REC', 'WAVELNTH']

In [17]:
wavelengths = [94, 
               131, 
               171, 
               193, 
               211, 
               304, 
               335]

In [19]:
aia_results = {}

for w in wavelengths:
    aia_results[str(w)] = Fido.search(
        times[0] | times[1] | times[2],
        a.jsoc.Notify("mgjeon@khu.ac.kr"),
        a.jsoc.Series("aia.lev1_euv_12s"),
        a.jsoc.Segment("image"),
        a.Wavelength(w*u.angstrom))

In [20]:
for key, value in aia_results.items():
    print(key, len(value))

94 3
131 3
171 3
193 3
211 3
304 3
335 3


In [21]:
for wavelnth, result in aia_results.items():
    Fido.fetch(result, path=f"download/12673/aia/{wavelnth}/" + "{file}")

Export request pending. [id=JSOC_20240405_001460, status=2]
Waiting for 0 seconds...
Export request pending. [id=JSOC_20240405_001460, status=1]
Waiting for 5 seconds...
Export request pending. [id=JSOC_20240405_001460, status=1]
Waiting for 5 seconds...
Export request pending. [id=JSOC_20240405_001460, status=1]
Waiting for 5 seconds...
Export request pending. [id=JSOC_20240405_001460, status=1]
Waiting for 5 seconds...
Export request pending. [id=JSOC_20240405_001460, status=1]
Waiting for 5 seconds...
Export request pending. [id=JSOC_20240405_001460, status=1]
Waiting for 5 seconds...
Export request pending. [id=JSOC_20240405_001460, status=1]
Waiting for 5 seconds...
Export request pending. [id=JSOC_20240405_001460, status=1]
Waiting for 5 seconds...
Export request pending. [id=JSOC_20240405_001460, status=1]
Waiting for 5 seconds...
Export request pending. [id=JSOC_20240405_001460, status=1]
Waiting for 5 seconds...
Export request pending. [id=JSOC_20240405_001460, status=1]
Waiti

Files Downloaded:   0%|          | 0/903 [00:00<?, ?file/s]

aia.lev1_euv_12s.2017-09-06T084001Z.94.image_lev1.fits:   0%|          | 0.00/6.77M [00:00<?, ?B/s]

aia.lev1_euv_12s.2017-09-06T084013Z.94.image_lev1.fits:   0%|          | 0.00/6.77M [00:00<?, ?B/s]

aia.lev1_euv_12s.2017-09-06T084025Z.94.image_lev1.fits:   0%|          | 0.00/6.77M [00:00<?, ?B/s]

...

aia.lev1_euv_12s.2017-09-07T150514Z.335.image_lev1.fits:   0%|          | 0.00/7.19M [00:00<?, ?B/s]

aia.lev1_euv_12s.2017-09-07T150526Z.335.image_lev1.fits:   0%|          | 0.00/7.19M [00:00<?, ?B/s]

aia.lev1_euv_12s.2017-09-07T150538Z.335.image_lev1.fits:   0%|          | 0.00/7.19M [00:00<?, ?B/s]

aia.lev1_euv_12s.2017-09-07T150550Z.335.image_lev1.fits:   0%|          | 0.00/7.19M [00:00<?, ?B/s]

aia.lev1_euv_12s.2017-09-07T150602Z.335.image_lev1.fits:   0%|          | 0.00/7.19M [00:00<?, ?B/s]